In [1]:
import cv2
import numpy as np
import os
import math
from numpy import linalg as LA

In [2]:
GOOD_MATCH_PERCENT = 0.15
RESULTS_DIR = "results/"

In [3]:
def get_files(dirpath, ext):
    files = [s for s in os.listdir(dirpath)
         if os.path.isfile(os.path.join(dirpath, s)) and os.path.splitext(s)[1] == ext]
    files.sort()
    return files

def createDir(dirpath, ext):
    if os.path.exists(dirpath):
        for file in get_files(dirpath, ext):
            os.remove(os.path.join(dirpath, file))
    else:
        os.mkdir(dirpath)

In [4]:
def align(table_file, pattern_im, table_im): #21/45
    sift = cv2.xfeatures2d.SIFT_create() 
    kp_image, desc_image =sift.detectAndCompute(pattern_im, None) 
    index_params = dict(algorithm = 1, trees = 5) 
    search_params = dict() 
    flann = cv2.FlannBasedMatcher(index_params, search_params) 

    frame = table_im
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 

    kp_grayframe, desc_grayframe = sift.detectAndCompute(grayframe, None) 

    matches= flann.knnMatch(desc_image, desc_grayframe, k=2) 

    good_points=[] 

    for m, n in matches: 

        if(m.distance < 0.6*n.distance): 
            good_points.append(m) 
            
    query_pts = np.float32([kp_image[m.queryIdx] 
                .pt for m in good_points]).reshape(-1, 1, 2) 
 
    train_pts = np.float32([kp_grayframe[m.trainIdx] 
                    .pt for m in good_points]).reshape(-1, 1, 2) 

    matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0) 

    matches_mask = mask.ravel().tolist() 
    
    height, width, channels = table_im.shape
    res = cv2.warpPerspective(frame, matrix, (width, height))
    
    pts = np.array([[13,400],[1330,606]], np.float32)
    pts = np.array([pts], np.float32)

    dst = cv2.perspectiveTransform(pts, matrix)
    
    res = cv2.rectangle(res, (int(dst[0][0][0]), int(dst[0][0][1])), 
                             (int(dst[0][1][0]), int(dst[0][1][1])), (0, 0, 255), 2)
#     cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", table_im)
    
    cv2.imwrite(RESULTS_DIR + table_file + "_align.jpg", res)

In [5]:
def alignImages(table_file, pattern_im, table_im): #37/48
    BLACK_LIMIT = 200
    
    im1Gray = cv2.cvtColor(pattern_im, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(table_im, cv2.COLOR_BGR2GRAY)
        
    orb = cv2.ORB_create(500, scaleFactor = 1.2, nlevels = 8,
                        edgeThreshold = 5, firstLevel = 0, WTA_K = 4, 
                        patchSize = 95, fastThreshold = 90)


    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    newkeypoints1 = []
    newdescriptors1 = []
    newkeypoints2 = []
    newdescriptors2 = []
    for index_obj in range(len(keypoints1)):
        point = keypoints1[index_obj]
        minDist = MIN_DIST
        minindex = 0
        for index_scene in range(len(keypoints2)):
            neighbour = keypoints2[index_scene]
            
            if ((neighbour.pt[0] - point.pt[0])**2 + (neighbour.pt[1] - point.pt[1])**2) < minDist:
                minDist = ((neighbour.pt[0] - point.pt[0])**2 + (neighbour.pt[1] - point.pt[1])**2)
                minindex = index_scene
                
        if minDist < MIN_DIST:
            newkeypoints1.append(point)
            newdescriptors1.append(descriptors1[index_obj])
            newkeypoints2.append(keypoints2[minindex])
            newdescriptors2.append(descriptors2[minindex])
        
    keypoints1 = newkeypoints1
    descriptors1 = np.array(newdescriptors1)
    keypoints2 = newkeypoints2
    descriptors2 = np.array(newdescriptors2)
    
    print(len(keypoints1), len(newkeypoints2), len(descriptors1), len(newdescriptors2))
    
    if len(keypoints1) < 4:
        print("Not enough elements")
        cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", table_im)
        return
    
    # Match features.
#     matcher = cv2.DescriptorMatcher_create("BruteForce")
    matcher = cv2.BFMatcher()
    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
#     numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
#     matches = matches[:numGoodMatches]

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)


    filter_matches = []
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[i].pt
        points2[i, :] = keypoints2[i].pt
#         points1[i, :] = keypoints1[match.queryIdx].pt
#         points2[i, :] = keypoints2[match.trainIdx].pt

        
    # Draw top matches
#     imMatches = cv2.drawMatches(pattern_im, keypoints1, table_im, keypoints2, matches, None)
    imMatches = cv2.drawMatches(im1Gray, keypoints1, im2Gray, keypoints2, matches, None)
    cv2.imwrite(RESULTS_DIR + table_file + "_matches.jpg", imMatches)
    
    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    if h is None:
        print("None matrix")
        cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", table_im)
        return
    
    height, width, channels = table_im.shape
    im1Reg = cv2.warpPerspective(pattern_im, h, (width, height))
    
    
    pts = np.array([[13,400],[1330,606]], np.float32)
    pts = np.array([pts], np.float32)
    
#     pattern_im = cv2.rectangle(pattern_im, (int(pts[0][0][0]), int(pts[0][0][1])), 
#                                (int(pts[0][1][0]), int(pts[0][1][1])), (0, 0, 255), 2)
#     cv2.imwrite(RESULTS_DIR + "pattern_border.jpg", pattern_im)

    dst = cv2.perspectiveTransform(pts, h)
    
    table_im = cv2.rectangle(table_im, (int(dst[0][0][0]), int(dst[0][0][1])), 
                             (int(dst[0][1][0]), int(dst[0][1][1])), (0, 0, 255), 2)
    cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", table_im)
    
#     cv2.imwrite(RESULTS_DIR + table_file + "_aligned.jpg", im1Reg)
    

In [6]:
def match(table_file, pattern_im, table_im): 
    BLACK_LIMIT = 170
    
    
#     im1Gray = cv2.cvtColor(pattern_im, cv2.COLOR_BGR2GRAY)
#     im2Gray = cv2.cvtColor(table_im, cv2.COLOR_BGR2GRAY)
    
#     im1Gray[im1Gray < BLACK_LIMIT] = 0
#     im1Gray[im1Gray > BLACK_LIMIT] = 255

#     im2Gray[im2Gray < BLACK_LIMIT] = 0
#     im2Gray[im2Gray > BLACK_LIMIT] = 255
    
    img_object = pattern_im
    img_scene = table_im

    #-- Step 1: Detect the keypoints using SURF Detector, compute the descriptors

#     detector = cv2.SIFT_create(minHessian)
    detector = cv2.SIFT_create(nfeatures=200, nOctaveLayers=5, 
                               contrastThreshold=0.04, edgeThreshold=10, 
                               sigma=1.6)
    keypoints_obj, descriptors_obj = detector.detectAndCompute(img_object, None)
    keypoints_scene, descriptors_scene = detector.detectAndCompute(img_scene, None)
    
    matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
    knn_matches = matcher.knnMatch(descriptors_obj, descriptors_scene, 2)

    ratio_thresh = RATION_TRESH
    good_matches = []
    for m,n in knn_matches:
        if m.distance < ratio_thresh * n.distance:
            good_matches.append(m)
            
    print(len(good_matches))

    img_matches = np.empty((max(img_object.shape[0], img_scene.shape[0]), img_object.shape[1]+img_scene.shape[1], 3), dtype=np.uint8)
    cv2.drawMatches(img_object, keypoints_obj, img_scene, keypoints_scene, good_matches, img_matches)
#     cv2.drawMatches(img_object, keypoints_obj, img_scene, keypoints_scene, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    #-- Localize the object
    obj = np.empty((len(good_matches),2), dtype=np.float32)
    scene = np.empty((len(good_matches),2), dtype=np.float32)
    for i in range(len(good_matches)):
        #-- Get the keypoints from the good matches
        obj[i,0] = keypoints_obj[good_matches[i].queryIdx].pt[0]
        obj[i,1] = keypoints_obj[good_matches[i].queryIdx].pt[1]
        scene[i,0] = keypoints_scene[good_matches[i].trainIdx].pt[0]
        scene[i,1] = keypoints_scene[good_matches[i].trainIdx].pt[1]
        
    if len(good_matches) < 4:
        print("    Lack of matches")
    else:
        H, _ =  cv2.findHomography(obj, scene, cv2.RANSAC, 3.0)
        
        if H is None:
            print("    None matrix")
        else:
            #-- Get the corners from the image_1 ( the object to be "detected" )
            obj_corners = np.empty((4,1,2), dtype=np.float32)
            obj_corners[0,0,0] = 0
            obj_corners[0,0,1] = 0
            obj_corners[1,0,0] = img_object.shape[1]
            obj_corners[1,0,1] = 0
            obj_corners[2,0,0] = img_object.shape[1]
            obj_corners[2,0,1] = img_object.shape[0]
            obj_corners[3,0,0] = 0
            obj_corners[3,0,1] = img_object.shape[0]
            scene_corners = cv2.perspectiveTransform(obj_corners, H)
            
            pts = np.array([[13,400],[1330,606]], np.float32)
            pts = np.array([pts], np.float32)

            dst = cv2.perspectiveTransform(pts, H)

            img_matches = cv2.rectangle(img_matches, 
                                        (int(dst[0][0][0]) + img_object.shape[1], int(dst[0][0][1])), 
                                        (int(dst[0][1][0]) + img_object.shape[1], int(dst[0][1][1])), 
                                        (0, 0, 255), 2)

            cv2.line(img_matches, (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])),\
                (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])), (0,255,0), 4)
            cv2.line(img_matches, (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])),\
                (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])), (0,255,0), 4)
            cv2.line(img_matches, (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])),\
                (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])), (0,255,0), 4)
            cv2.line(img_matches, (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])),\
                (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])), (0,255,0), 4)
    
    cv2.imwrite(RESULTS_DIR + table_file + "_matches.jpg", img_matches)

In [7]:
def largestTrianglePointsIdxs(points):
    area = 0
    pointsIdxs = [0,0,0]
    if len(points) < 3:
        return None
    
    for i in range(len(points)-2):
        x1,y1 = points[i]
        for j in range(i+1,len(points)-1):
            x2,y2 = points[j]
            for k in range(j+1,len(points)):
                x3,y3 = points[k]
                if abs(0.5*(x1*(y2-y3)+x2*(y3-y1)+x3*(y1-y2))) > area :
                    area = abs(0.5*(x1*(y2-y3)+x2*(y3-y1)+x3*(y1-y2)))
                    pointsIdxs = [i,j,k]

    return pointsIdxs

In [105]:
def alignImages(table_file, pattern_im, table_im, regions_pattern, regions_table):
    keypoints_pattern = regions_pattern
    keypoints_table = regions_table
    
    print(len(keypoints_pattern), len(keypoints_table))
    if len(keypoints_pattern) < 3 or len(keypoints_table) < 3:
        print("Not enough elements")
        cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", table_im)
        return

    pts1 = []
    pts2 = []
    new_matches = []
    for pIndex, keypoint_pattern in enumerate(keypoints_pattern):
        minDist = 35
        pairIndex = 0
        for tIndex, keypoint_table in enumerate(keypoints_table):
            if math.sqrt((keypoint_pattern[0] - keypoint_table[0])**2 + (keypoint_pattern[1] - keypoint_table[1])**2) < minDist:
                if keypoint_table not in pts2:
                    pairIndex = tIndex
                    minDist = math.sqrt((keypoint_pattern[0] - keypoint_table[0])**2 
                                      + (keypoint_pattern[1] - keypoint_table[1])**2)
        if minDist < 35:
            keypoint_table = keypoints_table[pairIndex]
            dmatch = cv2.DMatch(pIndex, pairIndex, 100)
            cv2.circle(table_im, (int(keypoint_pattern[0]), int(keypoint_pattern[1])), 
                       radius=35, color=(255, 0, 0), thickness=2)
            new_matches.append(dmatch)
            pts1.append(keypoint_pattern)
            pts2.append(keypoint_table)
        
    pts1 = np.float32(pts1)
    pts2 = np.float32(pts2)
    
    for point in keypoints_pattern:
        cv2.circle(pattern_im, (int(point[0]), int(point[1])), radius=10, color=(0, 0, 255), thickness=-1)
        
    for point in pts1.astype(int):
        cv2.circle(pattern_im, (int(point[0]), int(point[1])), radius=10, color=(0, 255, 0), thickness=-1)
        
    
    for point in keypoints_pattern:
        cv2.circle(table_im, (int(point[0]), int(point[1])), radius=10, color=(255, 0, 0), thickness=-1)
        
    for point in keypoints_table:
        cv2.circle(table_im, (int(point[0]), int(point[1])), radius=10, color=(0, 0, 255), thickness=-1)
        
    for point in pts2.astype(int):
        cv2.circle(table_im, (int(point[0]), int(point[1])), radius=10, color=(0, 255, 0), thickness=-1)
        
    kp_pattern = [cv2.KeyPoint(point[0], point[1], 5) for point in keypoints_pattern]
    kp_table = [cv2.KeyPoint(point[0], point[1], 5) for point in keypoints_table]
    
    imMatches = cv2.drawMatches(pattern_im, kp_pattern, table_im, kp_table, new_matches, None)          
#     cv2.imwrite(RESULTS_DIR + table_file + "_matches.jpg", imMatches)
    
    print("Matches:", len(new_matches))
    if len(new_matches) < 3:
        print("Lack of matches")
        cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", table_im)
        return 

    pointsIndexes = largestTrianglePointsIdxs(pts1)
    
    pts1 = np.float32([pts1[pointsIndexes[0]], pts1[pointsIndexes[1]], pts1[pointsIndexes[2]]])
    pts2 = np.float32([pts2[pointsIndexes[0]], pts2[pointsIndexes[1]], pts2[pointsIndexes[2]]])
    
    matrixAff = cv2.getAffineTransform(pts1, pts2) 
    
    pts = np.array([[13,400],[1330,606]], np.float32)
    pts = np.array([pts], np.float32)
    
    dst = cv2.transform(pts, matrixAff).astype(int)
    
    pts_for_affin =  np.array([pts1], np.float32)
    dst_points = cv2.transform(pts_for_affin, matrixAff).astype(int)
                
    if abs(len(keypoints_pattern) - len(keypoints_table)) < len(keypoints_pattern) * 0.15 \
        and len(new_matches) > len(keypoints_table) * 0.85:
        
        for point in dst_points[0]:
            cv2.circle(table_im, (int(point[0]), int(point[1])), radius=10, color=(255, 0, 255), thickness=-1)
        
        cv2.line(table_im, dst_points[0][0], dst_points[0][1], 
                 (255, 0, 255), 3, cv2.LINE_AA)
        cv2.line(table_im, dst_points[0][1], dst_points[0][2], 
                 (255, 0, 255), 3, cv2.LINE_AA)
        cv2.line(table_im, dst_points[0][2], dst_points[0][0], 
                 (255, 0, 255), 3, cv2.LINE_AA)

        table_im = cv2.rectangle(table_im, pts[0][0].astype(int), 
                                 pts[0][1].astype(int), (0, 0, 255), 5)
        table_im = cv2.rectangle(table_im, dst[0][0], dst[0][1], (0, 255, 0), 5)
        
        print("Correct pattern!")
    else:
        print("Another pattern!")

    cv2.imwrite(RESULTS_DIR + table_file + "_borders.jpg", table_im)
    

In [106]:
def rotateIMG(image):
#     image = cv2.imread(jpgPath)
    midImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    BLACK_LIMIT = 200
    
    midImage[midImage < BLACK_LIMIT] = 0
    midImage[midImage > BLACK_LIMIT] = 255

    thresh = cv2.threshold(midImage, 190, 255, cv2.THRESH_BINARY_INV)[1]

    midImage = np.copy(thresh)
    
#     cv2.imwrite(rotateJpgPath[:-4] + "_thresh.jpg", midImage, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
    
    dstImage = cv2.Canny(midImage, 80, 200, 3)
    
#     cv2.imwrite(rotateJpgPath[:-4] + "_canny.jpg", dstImage, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
    
    lineimage = image.copy()
 
    lines = cv2.HoughLines(dstImage, 1, np.pi/180, 375)
    
    approvedCount = 0
    
    verticalCount = 0
    verticalSum = 0
    
    horizontalCount = 0
    horizontalSum = 0
    
    
    if lines is None:
        print("No lines")
        return
    
    for i in range(len(lines)):
        for rho, theta in lines[i]:
            lineAngle = theta / np.pi * 180
            
            if (lineAngle > 180):
                print(lineAngle)
                
            if (lineAngle < 80 or (lineAngle > 100 and lineAngle < 170) \
                or (lineAngle > 190 and lineAngle < 260) or lineAngle > 280):
                continue
            else:
                horizontalCount += 1
                horizontalSum += lineAngle   
                
                            
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho
                x1 = int(round(x0 + 4000 * (-b)))
                y1 = int(round(y0 + 4000 * a))
                x2 = int(round(x0 - 4000 * (-b)))
                y2 = int(round(y0 - 4000 * a))
#                 cv2.line(lineimage, (x1, y1), (x2, y2), (0, 0, 255), 1, cv2.LINE_AA)


    average = horizontalSum / horizontalCount - 90 if horizontalCount > 0 else 0
    
    rotate = lineimage
        
    h, w = rotate.shape[:2]
    RotateMatrix = cv2.getRotationMatrix2D((w/2.0, h/2.0), average, 1)
    rotateImg = cv2.warpAffine(rotate, RotateMatrix, (w, h), borderValue=(255, 255, 255))

#     cv2.imwrite(rotateJpgPath, rotate, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
    return rotateImg

In [107]:
def line_intersection(l1, l2):
    line1 = ([l1[0],l1[1]],[l1[2],l1[3]])
    line2 = ([l2[0],l2[1]],[l2[2],l2[3]])
    
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return (int(x), int(y))

def distance(point1, point2):
    return math.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)

def isPontOnLine(point, line):
    return abs(distance([line[0],line[1]], point) + distance([line[2],line[3]], point) \
        - distance([line[0],line[1]], [line[2],line[3]])) < 3


def getKeyPointsRegions(rotate_img, table_file):
    midImage = cv2.cvtColor(rotate_img, cv2.COLOR_BGR2GRAY)
            
    BLACK_LIMIT = 200
    
    midImage[midImage < BLACK_LIMIT] = 0
    midImage[midImage > BLACK_LIMIT] = 255

    thresh = cv2.threshold(midImage, 190, 255, cv2.THRESH_BINARY_INV)[1]

    horizontal = np.copy(thresh)
    vertical = np.copy(thresh)
    
    scale = 10
        
    horizontalCols = horizontal.shape[1]
    horizontalSize = horizontalCols / scale
    horizontalSize = int(horizontalSize)
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontalSize, 1))
    horizontal = cv2.morphologyEx(horizontal, cv2.MORPH_OPEN, horizontalStructure)

    
    verticalRows = vertical.shape[0]
    verticalSize = verticalRows / scale
    verticalSize = int(verticalSize)
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalSize))
    vertical = cv2.morphologyEx(vertical, cv2.MORPH_OPEN, verticalStructure)
    
    
    SE = cv2.getStructuringElement(cv2.MORPH_RECT, (1,3))
    horizontal = cv2.dilate(horizontal, SE, iterations=3)

    SE = cv2.getStructuringElement(cv2.MORPH_RECT, (5,1))
    vertical = cv2.dilate(vertical, SE, iterations=5)
    
    lines_mask = horizontal + vertical
    
    
    points_image = rotate_img.copy()
    
    linesP = cv2.HoughLinesP(lines_mask, 1, np.pi/180, 150, None, 50, 20)
    

    if linesP is not None:    
        horizontalLines = [line[0] for line in linesP if abs(line[0][2] - line[0][0]) > abs(line[0][3] - line[0][1])]
        verticalLines = [line[0] for line in linesP if abs(line[0][2] - line[0][0]) < abs(line[0][3] - line[0][1])]
        
        count = 0
        for i in range(0, len(horizontalLines)):
            l = horizontalLines[i]
            l[0] -= 10
            l[2] += 10
            horizontalLines[i] = l
#             lineimg = cv2.line(points_image, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 2, cv2.LINE_AA)
#             cv2.imwrite(RESULTS_DIR + table_file + str(count) + "_points.jpg", lineimg)
            count += 1
            
        for i in range(0, len(verticalLines)):
            l = verticalLines[i]
            l[1] += 10
            l[3] -= 10
            verticalLines[i] = l
#             lineimg = cv2.line(points_image, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 2, cv2.LINE_AA)
#             cv2.imwrite(RESULTS_DIR + table_file + str(count) + "_points.jpg", lineimg)
            count += 1
            
#         print(count)
        keypoints = []
        count = 0
        for horizontal_line in horizontalLines:
            for vertical_line in verticalLines:
                point = line_intersection(horizontal_line, vertical_line)
                if point is not None:
                    if isPontOnLine(point, vertical_line) and isPontOnLine(point, horizontal_line):
                        keypoints.append(point)
                    
        hasChanges = True
        while hasChanges:
            hasChanges = False
            
            for keypoint in keypoints:
                if hasChanges:
                    break
                neighbours = []
                for neighbour in keypoints:
                    if keypoint == neighbour:
                        continue
                        
                    if math.sqrt((keypoint[0]-neighbour[0])**2 + (keypoint[1]-neighbour[1])**2) < 30:
                        neighbours.append(neighbour)

                if len(neighbours) > 0:
                    new_keypoint = keypoint
                    for neighbour in neighbours:
                        new_keypoint = (new_keypoint[0] + neighbour[0], 
                                        new_keypoint[1] + neighbour[1])
                    
                    new_keypoint = (int(new_keypoint[0] / (len(neighbours) + 1)), 
                                    int(new_keypoint[1] / (len(neighbours) + 1)))

                    keypoints = [point for point in keypoints if point not in neighbours and point != keypoint]
                    keypoints.append(new_keypoint)
                    
                    hasChanges = True
                    break
                        
        for point in keypoints:
            cv2.circle(points_image, point, radius=10, color=(0, 255, 0), thickness=-1)
                    
#         cv2.imwrite(RESULTS_DIR + table_file + "_points.jpg", points_image)
        
        return keypoints

In [108]:
createDir(RESULTS_DIR, '.jpg')
PATTERN_NAME = "pattern_3.jpg"
# PATTERN_NAME = "form.jpg"
  
pattern_im = cv2.imread(PATTERN_NAME, cv2.IMREAD_COLOR)
rotatePatternPath = RESULTS_DIR + PATTERN_NAME[:-4] + "_r.jpg"
rotate_pattern = rotateIMG(pattern_im)
cv2.imwrite(rotatePatternPath, rotate_pattern)
regions_pattern = getKeyPointsRegions(rotate_pattern, PATTERN_NAME)
    
RATION_TRESH = 0.7
for table_file in get_files("tables/", ".jpg"):

#     if table_file != "33_311-1021-001_1.jpg":
#         continue
    
    print(table_file)
    
    table_path = "tables/" + table_file
    rotateJpgPath = RESULTS_DIR + table_file[:-4] + "_r.jpg"

    table_im = cv2.imread(table_path, cv2.IMREAD_COLOR)

    rotate_table = rotateIMG(table_im)
    regions_table = getKeyPointsRegions(rotate_table, table_file)
    
    if regions_table is not None:
        alignImages(table_path.split("/")[-1].split(".")[0], rotate_pattern.copy(), rotate_table.copy(), 
                    regions_pattern.copy(), regions_table.copy())

#     align(table_path.split("/")[-1].split(".")[0], pattern_im, table_im)
#     match(table_path.split("/")[-1].split(".")[0], pattern_im, table_im)

#     break

33_131-110-1010-001_1.jpg
61 0
Not enough elements
33_311-1021-001_1.jpg
61 65
Matches: 61
Correct pattern!
33_311-1022-001_1.jpg
61 63
Matches: 60
Correct pattern!
33_311-1821-001_1.jpg


/Users/titrom/miniconda3/envs/keyValEnv/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in int_scalars
  if __name__ == '__main__':


61 79
Matches: 36
Another pattern!
33_641-1000-001_1.jpg
61 62
Matches: 61
Correct pattern!
33_641-1001-001_1.jpg
61 62
Matches: 61
Correct pattern!
33_641-1010-001_1.jpg
61 64
Matches: 56
Correct pattern!
33_641-1020-001_1.jpg
61 69
Matches: 48
Another pattern!
33_641-1021-001_1.jpg
61 62
Matches: 61
Correct pattern!
33_641-1023-001_1.jpg
61 62
Matches: 59
Correct pattern!
33_641-1024-001_1.jpg
33_641-1040-001_1.jpg
61 75
Matches: 28
Another pattern!
33_641-1041-001_1.jpg
61 55
Matches: 51
Correct pattern!
33_641-1042-001_1.jpg
61 63
Matches: 61
Correct pattern!
33_641-1048-001_1.jpg
61 61
Matches: 61
Correct pattern!
33_641-1049-001_1.jpg
61 62
Matches: 61
Correct pattern!
33_641-1050-001_1.jpg
61 62
Matches: 61
Correct pattern!
33_641-1060-001_1.jpg
61 63
Matches: 60
Correct pattern!
33_641-1061-001_1.jpg
61 56
Matches: 54
Correct pattern!
33_641-1062-001_1.jpg
61 63
Matches: 61
Correct pattern!
33_641-10SK1-001_1.jpg
61 0
Not enough elements
33_641-130-1090-001_1.jpg
61 63
Matches:

In [12]:
# add middle of vertices